# Demo Setup 

In [ ]:
#
# Copyright (C) Analytics Engines 2021
# Alastair McKinley (a.mckinley@analyticsengines.com)
#

import requests
from rich import print
import arrow
import json

base_url = "http://127.0.0.1:3001/"

def login():
    r = requests.post("http://127.0.0.1:3001/rpc/login",json={"email" : "admin@beis.gov.uk", "password" : "Password1!"})
    jwt = r.json()["signed_jwt"]
    return jwt

def subscribe_to_events():
    jwt = login()
    headers = {
        'Authorization' : f'Bearer {jwt}',
        'Content-Type' : 'application/json',
        'Accept' : 'application/vnd.pgrst.object+json'
    }

    resp = requests.get(
        base_url + 'event_type',
        headers=headers,
        params={
            "event_name" : "eq.new_document_revision"
        }
    )

    resp.raise_for_status()

    event_type_id = resp.json()['id']

    headers = {
        'Authorization' : f'Bearer {jwt}',
        'Content-Type' : 'application/json',
        'Prefer': 'resolution=merge-duplicates'
    }

    resp = requests.post(
        base_url + 'event_subscription',
        headers=headers,
        json = {
            "event_type_id" : event_type_id,
            "event_filters" : [
                {
                    "event_key" : "document_pk",
                    "event_filter" : "/uksi/2002/618"
                }
            ],
            "deliver_async" : True
        },
        params = {
            "on_conflict" : "event_type_id,event_filters,user_id"
        }
    )

    resp.raise_for_status()

    headers = {
        'Authorization' : f'Bearer {jwt}',
        'Content-Type' : 'application/json',
        'Accept' : 'application/vnd.pgrst.object+json'
    }

    resp = requests.get(
        base_url + 'event_type',
        headers=headers,
        params={
            "event_name" : "eq.stale_document_relationship"
        }
    )

    headers = {
        'Authorization' : f'Bearer {jwt}',
        'Content-Type' : 'application/json',
        'Prefer': 'resolution=merge-duplicates'
    }

    event_type_id = resp.json()['id']

    resp = requests.post(
        base_url + 'event_subscription',
        headers=headers,
        json = {
            "event_type_id" : event_type_id,
            "event_filters" : [
                {
                    "event_key" : "changed_document_pk",
                    "event_filter" : "/uksi/2002/618"
                }
            ],
            "deliver_async" : True
        },
        params = {
            "on_conflict" : "event_type_id,event_filters,user_id"
        }
    )

    resp.raise_for_status()

def check_event_subscriptions():
    jwt = login()
    headers = {
        'Authorization' : f'Bearer {jwt}',
        'Content-Type' : 'application/json'
    }

    resp = requests.get(
        base_url + 'event_subscription_stream',
        headers=headers
    )

    resp.raise_for_status()

    print(resp.json())

def do_graph_search(jwt,all_rels=False,id=0):
    headers={
        "Content-Type" : "application/json",
        # "Accept": "application/vnd.pgrst.object+json",
        "Authorization": f"Bearer {jwt}"

    }

    json_data = {
        "filters" : [{
            "operator" : "and",
            "filter_elements" : [{
                "document_metadata_category": "title",
                "websearch_tsquery" : "medical"
            }]
        }],
        "relationship_names" : ([] if all_rels else ['guidance_references_legislation']),
        "metadata_categories" : ['title']
    }
    resp = requests.post(
        'http://127.0.0.1:3001/rpc/graph_search',
        json=json_data,
        headers=headers
    )
    resp.raise_for_status()
    dump = json.dumps({
        "body":json_data,
        "headers":headers,
        "response":resp.json()
    })

    with open(f'graph_query_{id}.json',"w") as f:
        f.write(dump)

    data = {}

    nodes = [n for n in resp.json()['nodes']]
    edges = []


    for e in resp.json()['edges']:
        if e['data']['relationship_confirmation']['confirmation_status'] == 'reconfirmed':
            e['data']['tooltip'] = f"""
                Previously stale relationship reconfirmed by user {e['data']['relationship_confirmation']['user_id']} 
                {arrow.get(e['data']['relationship_confirmation']['confirmed_on']).humanize()}
            """
        elif e['data']['stale'] == 'True':
            e['data']['tooltip'] = f"Stale relationship"
        
        if e['data']['property_key'] == 'same_named_entity':
            e['data']['tooltip'] = f"{e['data']['properties'][e['data']['property_key']]}"
        
        edges.append(e)
        

    data['nodes'] = nodes
    data['edges'] = edges

    return data

def get_db_conn():
    import psycopg2
    db_conn = psycopg2.connect(dbname="orp_alpha", user="postgres", password="admin", port=5435, host="127.0.0.1")
    return db_conn

def do_document_update():
    db_conn = get_db_conn()
    with db_conn:
        with db_conn.cursor() as curs:
            curs.execute("""
                select count(*) from public_api.document where pk = '/uksi/2002/618'
            """)

            res = curs.fetchone()

            if res[0] == 2:
                print("skipping test as event already created")
                do_logs_check = False
            else:
                curs.execute("""select load_single_document(
                    (select id from public_api.document_type where name = 'legislation.gov.uk'),
                    '/demo_data/revisions/618_v2.xml'
                );

                select update_document_metadata(id) from public_api.document_metadata_definition;

                select update_document_graph_relationship_definition((select id from public_api.document_graph_relationship_definition where name = 'guidance_references_legislation'));
                """)
    db_conn.close()
    

def setup_cygraph():
    import ipycytoscape as cy
    cyGraph = cy.CytoscapeWidget()

    cyGraph.set_style([
        {
            "selector" : 'node',
            "style" : {
                'label': 'data(title)',
                'font-size': '8px',
                "height":40,
                "width":40
            }
        },
        {
            "selector" : 'node[document_type_name=\"legislation.gov.uk\"]',
            "style" : {
                "background-color":"#30c9bc"
            }
        },
        {
            "selector" : 'node[document_type_name=\"orpml\"]',
            "style" : {
                "background-color":"#11479e"
            }
        },
        {
            "selector" : 'edge',
            "style" : {
                'font-size': '8px'
            }
        },
        {
            "selector" : 'edge[stale=\"False\"]',
            "style" : {
                "opacity": "1"
            }
        },
        {
            "selector" : 'edge[stale=\"True\"]',
            "style" : {
                "opacity": "0.5"
            }
        },
        {
            "selector" : 'edge[property_key=\"guidance_references_legislation\"]',
            "style" : {
                "label": "References Legislation",
                "line-color": "#d0b7d5"
            }
        },
        {
            "selector" : 'edge[property_key=\"same_named_entity\"]',
            "style" : {
                "label": "Same Named Entity",
                "line-color": "blue"
            }
        }
    ])

    cyGraph.set_layout(name = 'cola',
        nodeSpacing = 150,
        edgeLengthVal = 45,
        animate = True,
        randomize = False,
        maxSimulationTime = 1500
    )

    cyGraph.layout.height = '700px'
    cyGraph.set_tooltip_source('tooltip')
    return cyGraph

def do_document_cleanup():
    db_conn = get_db_conn()
    with db_conn:
        with db_conn.cursor() as curs:
            curs.execute("""
                delete from public_api.document where pk = '/uksi/2002/618' and latest is true and revision_number = 1;
                delete from public_api.event;
                delete from public_api.event_stream;
            """)
    db_conn.close()

do_document_cleanup()

def reconfirm_edge(document_graph_id):
    # mark the edge as reconfirmed
    headers = {
        'Authorization' : f'Bearer {jwt}',
        'Content-Type' : 'application/json'
    }

    resp = requests.post(
        base_url + 'document_graph_relationship_confirmation',
        headers=headers,
        json = {
            "document_graph_id" : document_graph_id,
            "confirmation_status" : "reconfirmed"
        }
    )

    resp.raise_for_status()

# Perform a graph search for Medical in the title with relationships of all types

In [ ]:
cyGraph = setup_cygraph()

jwt = login()

resp = do_graph_search(jwt,all_rels=True,id=1)

# print(resp)

cyGraph.graph.add_graph_from_json(resp)

cyGraph


# Perform A Graph Search For Documents with "Medical" in the Title and related guidance
This will return documents and related guidance documents

In [ ]:

cyGraph = setup_cygraph()

jwt = login()

resp = do_graph_search(jwt,id=2)

# print(resp.json())

cyGraph.graph.add_graph_from_json(resp)

cyGraph


# A user will subscribe to changes to this legislation and it's related documents

In [ ]:

subscribe_to_events()



# Simulate the creation of a new revision of the legislation (this will indicate that the link to guidance may be "Stale")

In [ ]:
do_document_update()

# Perform the same search after the legislation update

In [ ]:
import requests
from rich import print
import ipycytoscape as cy

cyGraph = setup_cygraph()

resp = do_graph_search(jwt,id=3)

# print(resp)

cyGraph.graph.add_graph_from_json(resp)

cyGraph

# Simulate a user checking their subscriptions (two notifications related to the subscribed legislation are present)

In [ ]:
check_event_subscriptions()

# Simulate a document owner indicating that the link from guidance to the primary legislation is still valid and does not need a new revision of the guidance

In [ ]:
import ipycytoscape as cy
jwt = login()
resp = do_graph_search(jwt,id=4)

cyGraph = setup_cygraph()


document_graph_id = resp['edges'][0]['data']['id']

reconfirm_edge(document_graph_id)

resp = do_graph_search(jwt,id=5)

# print(resp)

cyGraph.graph.add_graph_from_json(resp)

cyGraph
